# Ejercicio Data Quality - Perfilado
## Evaluar la calidad de datos de las ventas de productos

Se quiere hacer una evaluación de calidad de datos sobre las ventas (sales) y pagos (payments). Para ello se requiere hacer un análisis de los siguientes puntos:
- Calidad de los datos
- Selección de clave principal
- Identificación de cardinalidad
- Obtener media, varianza y desviacion Estandar, covarianza, correlacion
- Mejorar la calidad.

**Referencia**: “Estadística Descriptiva con Python y Pandas”: https://coderhook.github.io/Descriptive%20Statistics

- Columnas sales:, orderNumber, orderLineNumber, orderDate, shippedDate, requiredDate, customerNumber, employeeNumber, productCode, status, comments, quantityOrdered, priceEach, sales_amount, origin

- Columnas payments:, customerNumber, checkNumber, paymentDate, amount

In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

## Cargar archivos

In [2]:
sales_df = pd.read_csv('../Pandas/datos/company_sales/sales.csv')

In [3]:
payments_df = pd.read_csv('../Pandas/datos/company_sales/payments.csv')

In [4]:
# Mostrar las primeras filas de cada dataset
sales_df.head(), payments_df.head()

(       0  0.1  0000-00-00 0000-00-00.1 0000-00-00.2  0.2   0.3 productCode  \
 0  10100    1  0000-00-00   0000-00-00   0000-00-00  363  1216    S24_3969   
 1  10100    2  0000-00-00   0000-00-00   0000-00-00  363  1216    S18_2248   
 2  10100    3  0000-00-00   0000-00-00   0000-00-00  363  1216    S18_1749   
 3  10100    4  0000-00-00   0000-00-00   0000-00-00  363  1216    S18_4409   
 4  10101    1  0000-00-00   0000-00-00   0000-00-00  128  1504    S18_2795   
 
     status                comments  0.4    0.00   0.00.1 origin  
 0  Shipped                     NaN   49   35.29  1729.21  spain  
 1  Shipped                     NaN   50   55.09  2754.50  spain  
 2  Shipped                     NaN   30  136.00  4080.00  spain  
 3  Shipped                     NaN   22   75.46  1660.12  spain  
 4  Shipped  Check on availability.   26  167.06  4343.56  spain  ,
      0 checkNumber  0000-00-00      0.00
 0  103    HQ336336  2004-10-19   6066.78
 1  103    JM555205  2003-06-05  1457

In [5]:
# Renombrar columnas con nombres más adecuados para sales.csv
sales_df.columns = [
"orderNumber", "orderLineNumber", "orderDate", "shippedDate", "requiredDate",
"customerNumber", "EmployeeNumber", "productCode", "status", "comments",
"quantityOrdered", "priceEach", "sales_amount", "origin"
]
# Renombrar columnas para payments.csv
payments_df.columns = ["customerNumber", "checkNumber", "paymentDate", "amount"]

sales_df.head(), payments_df.head()

(   orderNumber  orderLineNumber   orderDate shippedDate requiredDate  \
 0        10100                1  0000-00-00  0000-00-00   0000-00-00   
 1        10100                2  0000-00-00  0000-00-00   0000-00-00   
 2        10100                3  0000-00-00  0000-00-00   0000-00-00   
 3        10100                4  0000-00-00  0000-00-00   0000-00-00   
 4        10101                1  0000-00-00  0000-00-00   0000-00-00   
 
    customerNumber  EmployeeNumber productCode   status  \
 0             363            1216    S24_3969  Shipped   
 1             363            1216    S18_2248  Shipped   
 2             363            1216    S18_1749  Shipped   
 3             363            1216    S18_4409  Shipped   
 4             128            1504    S18_2795  Shipped   
 
                  comments  quantityOrdered  priceEach  sales_amount origin  
 0                     NaN               49      35.29       1729.21  spain  
 1                     NaN               50     

In [6]:
# Revisar tipos de datos
sales_info = sales_df.dtypes
payments_info = payments_df.dtypes
sales_info, payments_info

(orderNumber          int64
 orderLineNumber      int64
 orderDate           object
 shippedDate         object
 requiredDate        object
 customerNumber       int64
 EmployeeNumber       int64
 productCode         object
 status              object
 comments            object
 quantityOrdered      int64
 priceEach          float64
 sales_amount       float64
 origin              object
 dtype: object,
 customerNumber      int64
 checkNumber        object
 paymentDate        object
 amount            float64
 dtype: object)

In [33]:
# Datos vacios
print('Datos vacios en sales.csv\n',sales_df.isna().sum())
print('\nDatos vacios en payments.csv\n',payments_df.isna().sum())

Datos vacios en sales.csv
 orderNumber           0
orderLineNumber       0
orderDate             0
shippedDate         142
requiredDate          0
customerNumber        0
EmployeeNumber        0
productCode           0
status                0
comments           2242
quantityOrdered       0
priceEach             0
sales_amount          0
origin                0
dtype: int64

Datos vacios en payments.csv
 customerNumber    0
checkNumber       0
paymentDate       0
amount            0
dtype: int64


In [34]:
# Verificamos fechas únicas en sales
print('\norderDate:',sales_df['orderDate'].unique())
print('\nshippedDate:',sales_df['shippedDate'].unique())
print('\nrequiredDate:',sales_df['requiredDate'].unique())


orderDate: ['0000-00-00' '2038-09-00']

shippedDate: ['0000-00-00' nan '2038-00-06' '2038-09-07']

requiredDate: ['0000-00-00' '2038-00-08' '2038-09-07']


In [35]:
# Contamos cuantas fechas hay de cada fecha única
print('orderDate:', sales_df['orderDate'].value_counts())
print('\nshippedDate:', sales_df['shippedDate'].value_counts())
print('\nrequiredDate:', sales_df['requiredDate'].value_counts())

orderDate: orderDate
0000-00-00    2998
2038-09-00       3
Name: count, dtype: int64

shippedDate: shippedDate
0000-00-00    2839
2038-00-06      17
2038-09-07       3
Name: count, dtype: int64

requiredDate: requiredDate
0000-00-00    2981
2038-00-08      17
2038-09-07       3
Name: count, dtype: int64


In [ ]:
# Verificamos comments únicos en sales
print('\ncomments:',sales_df['comments'].unique())

In [ ]:
# Contamos comments hay de cada comments único
print('comments:', sales_df['comments'].value_counts())

In [17]:
# Elimino en sales.csv columnas de fecha y comments ya que no tienen sentido

sales_df_clean = sales_df.drop(columns=['orderDate', 'shippedDate', 'requiredDate', 'comments'])
sales_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3001 entries, 0 to 3000
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   orderNumber      3001 non-null   int64  
 1   orderLineNumber  3001 non-null   int64  
 2   customerNumber   3001 non-null   int64  
 3   EmployeeNumber   3001 non-null   int64  
 4   productCode      3001 non-null   object 
 5   status           3001 non-null   object 
 6   quantityOrdered  3001 non-null   int64  
 7   priceEach        3001 non-null   float64
 8   sales_amount     3001 non-null   float64
 9   origin           3001 non-null   object 
dtypes: float64(2), int64(5), object(3)
memory usage: 234.6+ KB


In [37]:
# Comprobamos valores nulos en sales_df_clean
sales_df_clean.isna().sum()

orderNumber        0
orderLineNumber    0
customerNumber     0
EmployeeNumber     0
productCode        0
status             0
quantityOrdered    0
priceEach          0
sales_amount       0
origin             0
dtype: int64

In [28]:
# Convertir las columnas numéricas y fechas al tipo correcto
sales_df_clean["orderNumber"] = pd.to_numeric(sales_df_clean["orderNumber"], errors="coerce")
sales_df_clean["orderLineNumber"] = pd.to_numeric(sales_df_clean["orderLineNumber"], errors="coerce")
sales_df_clean["customerNumber"] = pd.to_numeric(sales_df_clean["customerNumber"], errors="coerce")
sales_df_clean["EmployeeNumber"] = pd.to_numeric(sales_df_clean["EmployeeNumber"], errors="coerce")
sales_df_clean["quantityOrdered"] = pd.to_numeric(sales_df_clean["quantityOrdered"], errors="coerce")
sales_df_clean["priceEach"] = pd.to_numeric(sales_df_clean["priceEach"], errors="coerce")
sales_df_clean["sales_amount"] = pd.to_numeric(sales_df_clean["sales_amount"], errors="coerce")

# sales_df_clean["orderDate"] = pd.to_datetime(sales_df_clean["orderDate"], errors="coerce")
# sales_df_clean["requiredDate"] = pd.to_datetime(sales_df_clean["requiredDate"], errors="coerce")
# sales_df_clean["shippedDate"] = pd.to_datetime(sales_df_clean["shippedDate"], errors="coerce")

payments_df["customerNumber"] = pd.to_numeric(payments_df["customerNumber"], errors="coerce")
payments_df["amount"] = pd.to_numeric(payments_df["amount"], errors="coerce")
payments_df["paymentDate"] = pd.to_datetime(payments_df["paymentDate"], errors="coerce")

# Mostrar la información corregida
sales_df_clean.info(), payments_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3001 entries, 0 to 3000
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   orderNumber      3001 non-null   int64  
 1   orderLineNumber  3001 non-null   int64  
 2   customerNumber   3001 non-null   int64  
 3   EmployeeNumber   3001 non-null   int64  
 4   productCode      3001 non-null   object 
 5   status           3001 non-null   object 
 6   quantityOrdered  3001 non-null   int64  
 7   priceEach        3001 non-null   float64
 8   sales_amount     3001 non-null   float64
 9   origin           3001 non-null   object 
dtypes: float64(2), int64(5), object(3)
memory usage: 234.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   customerNumber  278 non-null    int64         
 1   checkNumber 

(None, None)

In [55]:
# # Convertir fechas en sales.csv
# date_columns_sales = ["orderDate", "shippedDate", "requiredDate"]
# for col in date_columns_sales:
#     sales_df[col] = pd.to_datetime(sales_df[col], errors='coerce') # Convierte y pone NaT en valores inválidos

# Convertir fecha en payments.csv
# payments_df["paymentDate"] = pd.to_datetime(payments_df["paymentDate"], errors='coerce')

# Verificamos si hay alguna fecha nula
# missing_dates_sales = sales_df[date_columns_sales].isnull().sum()
missing_dates_payments = payments_df["paymentDate"].isnull().sum()

# missing_dates_sales, missing_dates_payments

print('\nFechas nulas en payments:',missing_dates_payments)


Fechas nulas en payments: 0


In [61]:
# Mostrar todas las filas duplicadas en sales_df_clean
print('\nSumatorio duplicados en sales', sales_df_clean.duplicated().sum())
sales_df_clean[sales_df_clean.duplicated(keep=False)]



Sumatorio duplicados en sales 5


,orderNumber,orderLineNumber,customerNumber,EmployeeNumber,productCode,status,quantityOrdered,priceEach,sales_amount,origin
27,10104,2,141,1370,S50_1514,Shipped,32,53.31,1705.92,spain
28,10104,2,141,1370,S50_1514,Shipped,32,53.31,1705.92,spain
2860,10410,2,357,1612,S18_3136,Shipped,34,84.82,2883.88,spain
2861,10410,2,357,1612,S18_3136,Shipped,34,84.82,2883.88,spain
2894,10413,6,175,1323,S32_3207,Shipped,24,56.55,1357.20,spain
2895,10413,6,175,1323,S32_3207,Shipped,24,56.55,1357.20,spain
2944,10419,1,382,1401,S18_1589,Shipped,37,100.80,3729.60,spain
2945,10419,1,382,1401,S18_1589,Shipped,37,100.80,3729.60,spain
2989,10425,3,119,1370,S18_2238,In Process,28,147.36,4126.08,spain
2990,10425,3,119,1370,S18_2238,In Process,28,147.36,4126.08,spain


In [62]:
# Mostrar todas las filas duplicadas en payments_df
print('\nSumatorio duplicados en payments', payments_df.duplicated().sum())
payments_df[payments_df.duplicated(keep=False)]


Sumatorio duplicados en payments 5


,customerNumber,checkNumber,paymentDate,amount
31,129,ID449593,2003-12-11,13923.93
32,129,ID449593,2003-12-11,13923.93
85,175,CITI3434344,2005-05-19,14500.78
86,175,CITI3434344,2005-05-19,14500.78
143,260,IO164641,2004-08-30,13527.58
144,260,IO164641,2004-08-30,13527.58
214,381,GB117430,2005-02-03,7379.90
215,381,GB117430,2005-02-03,7379.90
268,487,AH612904,2003-09-28,14997.09
269,487,AH612904,2003-09-28,14997.09


In [63]:
# Elimino filas duplicadas en sales_df_clean  y payments dejando la primera que aparece

sales_df_cleaned = sales_df_clean.drop_duplicates(keep='last')
payments_df_cleaned = payments_df.drop_duplicates(keep='last')

print('\nSumatorio duplicados en sales_df_cleaned', sales_df_cleaned.duplicated().sum())
print('\nSumatorio duplicados en payments', payments_df_cleaned.duplicated().sum())


Sumatorio duplicados en sales_df_cleaned 0

Sumatorio duplicados en payments 0


In [64]:
# Muestro la información de los df limpios

sales_df_cleaned.info(), payments_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2996 entries, 0 to 3000
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   orderNumber      2996 non-null   int64  
 1   orderLineNumber  2996 non-null   int64  
 2   customerNumber   2996 non-null   int64  
 3   EmployeeNumber   2996 non-null   int64  
 4   productCode      2996 non-null   object 
 5   status           2996 non-null   object 
 6   quantityOrdered  2996 non-null   int64  
 7   priceEach        2996 non-null   float64
 8   sales_amount     2996 non-null   float64
 9   origin           2996 non-null   object 
dtypes: float64(2), int64(5), object(3)
memory usage: 257.5+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 273 entries, 0 to 277
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   customerNumber  273 non-null    int64         
 1   checkNumber     273 no

(None, None)

In [81]:
# Calcular el sumatorio de valores para cada columna
sum_values_sales = sales_df_cleaned.count()

# Calcular el sumatorio de valores únicos para cada columna
unique_sum_sales = sales_df_cleaned.nunique()

# Crear un nuevo DataFrame para mostrar ambos sumatorios
result_df_sales = pd.DataFrame({
    'Sumatorio de valores': sum_values_sales,
    'Sumatorio de valores únicos': unique_sum_sales
})

print(result_df_sales)

                 Sumatorio de valores  Sumatorio de valores únicos
orderNumber                      2996                          326
orderLineNumber                  2996                           18
customerNumber                   2996                           98
EmployeeNumber                   2996                           15
productCode                      2996                          109
status                           2996                            6
quantityOrdered                  2996                           61
priceEach                        2996                         1573
sales_amount                     2996                         2885
origin                           2996                            2


In [82]:
# Calcular el sumatorio de valores para cada columna
sum_values_pay = payments_df_cleaned.count()

# Calcular el sumatorio de valores únicos para cada columna
unique_sum_pay = payments_df_cleaned.nunique()

# Crear un nuevo DataFrame para mostrar ambos sumatorios
result_df_pay = pd.DataFrame({
    'Sumatorio de valores': sum_values_pay,
    'Sumatorio de valores únicos': unique_sum_pay
})

print(result_df_pay)

                Sumatorio de valores  Sumatorio de valores únicos
customerNumber                   273                           98
checkNumber                      273                          273
paymentDate                      273                          232
amount                           273                          273


In [84]:
# Evaluar unicidad de posibles claves primarias en cada dataset

# Para sales.csv, posibles claves: orderNumber, orderLineNumber (combinación)
sales_unique_order = sales_df_cleaned["orderNumber"].nunique()
sales_total_rows = len(sales_df_cleaned)
sales_unique_combination = sales_df_cleaned[["orderNumber", "orderLineNumber"]].duplicated().sum() # Chequear duplicados

# Para payments.csv, posibles claves: checkNumber (supuestamente única)
payments_unique_check = payments_df_cleaned["checkNumber"].nunique()
payments_total_rows = len(payments_df_cleaned)
payments_duplicated_checks = payments_total_rows - payments_unique_check

print(sales_unique_order, sales_unique_combination, payments_unique_check,
payments_duplicated_checks)

326 0 273 0


In [ ]:
# Evaluar la relación entre sales y payments a través de customerNumber

# Contar clientes únicos en cada dataset
unique_customers_sales = sales_df["customerNumber"].nunique()
unique_customers_payments = payments_df["customerNumber"].nunique()

# Contar clientes comunes entre ambos datasets
common_customers = len(set(sales_df["customerNumber"]).intersection(set(payments_df["customerNumber"])))
unique_customers_sales, unique_customers_payments, common_customers

In [ ]:
# Calcular estadísticas descriptivas en variables numéricas
stats_sales = sales_df[["quantityOrdered", "priceEach", "sales_amount"]].describe().T
stats_payments = payments_df[["amount"]].describe().T

# Calcular varianza y desviación estándar
variance_sales = sales_df[["quantityOrdered", "priceEach", "sales_amount"]].var()
std_dev_sales = sales_df[["quantityOrdered", "priceEach", "sales_amount"]].std()
variance_payments = payments_df[["amount"]].var()
std_dev_payments = payments_df[["amount"]].std()

# Calcular covarianza entre variables en sales.csv
covariance_sales = sales_df[["quantityOrdered", "priceEach", "sales_amount"]].cov()

# Calcular correlación entre variables en sales.csv
correlation_sales = sales_df[["quantityOrdered", "priceEach", "sales_amount"]].corr()

# Mostrar los resultados en consola usando print
print("=== Estadísticas de Sales ===")
print(stats_sales)
print("\n=== Estadísticas de Payments ===")
print(stats_payments)
print("\n=== Covarianza en Sales ===")
print(covariance_sales)
print("\n=== Correlación en Sales ===")
print(correlation_sales)

In [ ]:
Aclean_sales_rows = len(sales_df)
Alean_payments_rows = len(payments_df)
Aclean_sales_rows, Alean_payments_rows

In [ ]:
# **Corrección de calidad de datos**

# 1. Eliminar duplicados en sales.csv (basado en orderNumber + orderLineNumber)
sales_df = sales_df.drop_duplicates(subset=["orderNumber", "orderLineNumber"])

# 2. Eliminar duplicados en payments.csv (basado en checkNumber)
payments_df = payments_df.drop_duplicates(subset=["checkNumber"])

# 3. Manejar valores nulos en las fechas de sales.csv
# - Se pueden eliminar registros con fechas nulas o imputarlas con una estrategia adecuada
sales_df = sales_df.dropna(subset=["orderDate", "shippedDate", "requiredDate"])

# Verificar nuevamente la cantidad de datos después de la limpieza
clean_sales_rows = len(sales_df)
clean_payments_rows = len(payments_df)
clean_sales_rows, clean_payments_rows